# Time Series Forecasting

## Setup

In [1]:
import os

In [2]:
import pandas as pd
import keras

import matplotlib.pyplot as plt
import seaborn as sns

## Climate Data Time Series

In [3]:
uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")

In [4]:
from zipfile import ZipFile

zip_file = ZipFile(zip_path)
zip_file.extractall(path="data/")
csv_path = "data/jena_climate_2009_2016.csv"

df = pd.read_csv(csv_path, usecols=["T (degC)"])

In [5]:
df

,T (degC)
0,-8.02
1,-8.41
2,-8.51
3,-8.31
4,-8.27
...,...
420546,-4.05
420547,-3.35
420548,-3.16
420549,-4.23


## Raw Data Visualization

In [6]:
# sns.lineplot(df)

## Data Preprocessing

Observation is recorded every 10 mins, that means 6 times per hour. We will resample one point per hour since no drastic change is expected within 60 minutes.

In [7]:
NUM_OBSERVATIONS = 2400

In [8]:
smaller_df = df.iloc[-NUM_OBSERVATIONS * 6::6, :].reset_index(drop=True)
smaller_df

,T (degC)
0,13.49
1,13.20
2,12.91
3,13.02
4,12.93
...,...
2395,-0.98
2396,-1.40
2397,-2.75
2398,-2.89


In [9]:
# sns.lineplot(smaller_df)

Since every feature has values with varying ranges, we do normalization to confine feature values to a range of $[0, 1]$ before training a neural network. We do this by subtracting the mean and dividing by the standard deviation of each feature.

In [10]:
def normalize(data):
    data_mean = data.mean(axis=0)
    data_std = data.std(axis=0)
    return (data - data_mean) / data_std

In [11]:
normalized_df = normalize(smaller_df)
normalized_df

,T (degC)
0,1.361937
1,1.310577
2,1.259218
3,1.278699
4,1.262760
...,...
2395,-1.200720
2396,-1.275102
2397,-1.514189
2398,-1.538983


In [12]:
# sns.lineplot(normalized_df)

The goal of the model will be to predict the temperature at the same time of the next day (i.e., 24 time steps in the future) given the past 5 days of data (i.e. 120 time steps).

In [13]:
FUTURE = 24
PAST = 120

In [14]:
# import numpy as np

# X = np.zeros((NUM_OBSERVATIONS - FUTURE - PAST, PAST))
# for i in range(NUM_OBSERVATIONS - FUTURE - PAST):
#     X[i, :] = np.array(normalized_df.iloc[i:i + PAST]).squeeze()
# X = X.reshape(-1, PAST, 1)

# y = np.array(normalized_df.iloc[PAST+FUTURE:]).squeeze()

In [15]:
import numpy as np

X = np.zeros((NUM_OBSERVATIONS - FUTURE - PAST, PAST, 3))
for i in range(NUM_OBSERVATIONS - FUTURE - PAST):
    for j in range(3):
        X[i, :, j] = np.array(normalized_df.iloc[i:i + PAST]).squeeze()
X = X.reshape(-1, PAST, 3)

y = np.array(normalized_df.iloc[PAST+FUTURE:]).squeeze()

Leave the last few entries for validation, and use the rest for training.

In [16]:
VAL_SPLIT = 0.1
train_count = int(len(X) * (1 - VAL_SPLIT))

X_train = X[:train_count]
X_val = X[train_count:]

y_train = y[:train_count]
y_val = y[train_count:]

In [17]:
print(X_train.shape)
print(X_val.shape)
print()
print(y_train.shape)
print(y_val.shape)

(2030, 120, 3)
(226, 120, 3)

(2030,)
(226,)


## Standard LSTM Model

### Creating the Model

In [18]:
# model = keras.Sequential(
#     layers = [
#         keras.layers.Input(shape=(PAST, 1)),
#         keras.layers.LSTM(32),
#         keras.layers.Dense(1)
#     ],
#     name="LSTM-Model"
# )
# model.compile(optimizer="adam", loss="mse")
# model.summary()

In [19]:
# es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=25, verbose=1)
# reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=1e-6, verbose=1)
# checkpointer = keras.callbacks.ModelCheckpoint(
#     monitor="val_loss",
#     filepath="models/time-series-forcasting-lstm.keras",
#     verbose=1,
#     save_best_only=True,
# )

In [20]:
# model.fit(
#     X_train,
#     y_train,
#     batch_size=128,
#     shuffle=True,
#     epochs=100,
#     validation_data=(X_val, y_val),
#     callbacks=[reduce_lr, es_callback, checkpointer]
# )

### Model Evaluation

In [21]:
# model_best = keras.models.load_model("models/time-series-forcasting-lstm.keras")

In [22]:
# model_best.evaluate(X_val, y_val)

In [23]:
def show_plot(plot_data, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "go", "rx"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(FUTURE, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (FUTURE + 5) * 2])
    plt.xlabel("Timestep")
    plt.show()
    return

In [24]:
# for x, y in list(zip(X_val, y_val))[10:10+3]:
#     show_plot(
#         [x[:, 0], y, model_best.predict(x.reshape(1, PAST, 1))[0]],
#         "Single Step Prediction",
#     )

## Standard GRU Model

### Creating the Model

In [25]:
# model = keras.Sequential(
#     layers = [
#         keras.layers.Input(shape=(PAST, 1)),
#         keras.layers.GRU(64),
#         keras.layers.Dense(1)
#     ],
#     name="GRU-Model"
# )
# model.compile(optimizer="adam", loss="mse")
# model.summary()

In [26]:
# es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=25, verbose=1)
# reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=1e-6, verbose=1)
# checkpointer = keras.callbacks.ModelCheckpoint(
#     monitor="val_loss",
#     filepath="models/time-series-forcasting-gru.keras",
#     verbose=1,
#     save_best_only=True,
# )

In [27]:
# model.fit(
#     X_train,
#     y_train,
#     batch_size=128,
#     shuffle=True,
#     epochs=100,
#     validation_data=(X_val, y_val),
#     callbacks=[reduce_lr, es_callback, checkpointer]
# )

### Model Evaluation

In [28]:
# model_best = keras.models.load_model("models/time-series-forcasting-gru.keras")

In [29]:
# model_best.evaluate(X_val, y_val)

In [30]:
# for x, y in list(zip(X_val, y_val))[10:10+3]:
#     show_plot(
#         [x[:, 0], y, model_best.predict(x.reshape(1, PAST, 1))[0]],
#         "Single Step Prediction",
#     )

## LRU Model

In [31]:
import keras_mml

In [32]:
# model = keras.Sequential(
#     layers = [
#         keras.layers.Input(shape=(PAST, 1)),
#         keras_mml.layers.LRUMML(64, 32),
#         keras.layers.Dense(1)
#     ],
#     name="LRU-Model"
# )
# model.compile(optimizer="adam", loss="mse")
# model.summary()

In [33]:
model = keras.Sequential(
    layers = [
        keras.layers.Input(shape=(PAST, 3)),
        keras_mml.layers.LRUMML(64, 32),
        keras.layers.Dense(1)
    ],
    name="LRU-Model"
)
model.compile(optimizer="adam", loss="mse")
model.summary()

Model: "LRU-Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lrumml (LRUMML)                 │ (None, 64)             │         4,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,897 (19.13 KB)

 Trainable params: 4,897 (19.13 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=20, verbose=1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=1e-6, verbose=1)
checkpointer = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath="models/time-series-forcasting-lru.keras",
    verbose=1,
    save_best_only=True,
)

In [35]:
model.fit(
    X_train,
    y_train,
    batch_size=128,
    shuffle=True,
    epochs=3,
    validation_data=(X_val, y_val),
    callbacks=[reduce_lr, es_callback, checkpointer]
)

Epoch 1/3


AttributeError: Exception encountered when calling LRUCellMML.call().

[1mDynamicJaxprTracer has no attribute values[0m

Arguments received by LRUCellMML.call():
  • inputs=jnp.ndarray(shape=(128, 3), dtype=float32)
  • states=['jnp.ndarray(shape=(128, 2, 32), dtype=float32)']
  • training=True

### Model Evaluation

In [ ]:
model_best = keras.models.load_model("models/time-series-forcasting-lru.keras")

In [ ]:
model_best.evaluate(X_val, y_val)

In [ ]:
# for x, y in list(zip(X_val, y_val))[10:10+3]:
#     show_plot(
#         [x[:, 0], y, model_best.predict(x.reshape(1, PAST, 1))[0]],
#         "Single Step Prediction",
#     )